In [ ]:
# init repo notebook
!git clone https://github.com/rramosp/ppdl.git > /dev/null 2> /dev/null
!mv -n ppdl/content/init.py ppdl/content/local . 2> /dev/null
!pip install -r ppdl/content/requirements.txt > /dev/null

# LAB 2. Distribution layers

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.integrate import quad
from progressbar import progressbar as pbar
from rlxutils import subplots, copy_func
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

%matplotlib inline

## Task 1: Compute probability for distribution layer output

**DO NOT** use Tensorflow

In [89]:
input_x = np.random.random(size=(3,5))+.1
print (input_x)

inp  = tf.keras.layers.Input(shape=(5,))
den1  = tf.keras.layers.Dense(10, activation="sigmoid", name="dense1", bias_initializer="glorot_uniform")(inp)
den2  = tf.keras.layers.Dense(2, activation="sigmoid", name="dense2", bias_initializer="glorot_uniform")(den1)
out  = tfp.layers.IndependentNormal(1)(den2)

m    = tf.keras.models.Model(inp, out)

In [90]:
d = m(input_x)
d

In [91]:
sample = np.random.random(size=[5] + d.batch_shape.as_list() + d.event_shape.as_list())
sample

In [92]:
dprobs = d.log_prob(sample).numpy()
dprobs

In [93]:
W1,b1 = m.get_layer("dense1").get_weights()
W2,b2 = m.get_layer("dense2").get_weights()
[i.shape for i in [W1,b1,W2,b2]]

In [94]:
sigm = lambda x: 1/(1+np.exp(-x))
params = sigm(sigm(input_x.dot(W1) + b1).dot(W2)+b2)
params

In [95]:
means = params[:,0]
stds  = np.log(np.exp(params[:,1])+1)
means, stds

In [96]:
distrs = [stats.norm(m,s) for m,s in zip(means, stds)]

In [97]:
# one way to solve it
probs = np.r_[[[np.log(distr.pdf(i)) for distr,i in zip(distrs,xi)] for xi in sample]][...,0]
print (np.allclose(probs, dprobs))
probs

In [84]:
# another way to solve it
probs = np.r_[[np.log(distr.pdf(sample[:,i,0])) for i,distr in enumerate(distrs)]].T
print (np.allclose(probs, dprobs))
probs

## Task 2: Compute probability with `event_shape`

**DO NOT** use Tensorflow

## Task 3: Compute gradients for distribution parameters

With negative log likelihood loss function.

**DO NOT** use Tensorflow

